In [2]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 17
### Team Member Names: Nelson Kang, Mane Muradyan, Phelan Niu
### Team Strategy Chosen: Market Beat

Goal: Market beat
Strategy: Sortino-centered stock selection with some short-term momentum tilt

Inputs:
- CSV file with tickers

Outputs:
- 

Game Plan:
1. load csv and set constants
2. get FX rate and static info of stocks
3. check eligibility of stocks (liquidity rule etc.)
4. Compute Sortino and momentum tilt and combine for a score
5. construct portfolio with cap and sector mix
6. Distribute stock weight


In [3]:
#Imports
import math
import time
import random
import numpy as np
import pandas as pd
import yfinance as yf
import os


# Defining most constants
N_min = 10 # min number of stocks in portfolio
N_max = 25 # max number of stocks in portfolio
max_weight = 0.15 # max weight per stock
sector_cap = 0.4 # max weight per sector
budget = 1_000_000 
MIN_avg_daily_volume = 5000
LIVE_date = "2025-11-21"
END_date = "2025-11-28"

fee_flat_usd = 2.15
fee_per_share_usd = 0.001


LOOK_BACK_START = "2022-01-01"
LOOK_BACK_END = "2025-08-08"



In [ ]:
#HELPERS
def get_tickers(file_name):
    """
    Input: local name of tickers csv file
    Output: list of tickers
    """
    df = pd.read_csv(file_name)
    first_col = df.columns[0]
    tickers = (df[first_col].str.strip().tolist())
    return tickers

def get_exchange_rate():
    """
    Input: None
    Output: Float
    Returns latest exchange rate CAD --> USD
    """
    fx = yf.Ticker("CADUSD=X")
    return float(fx.fast_info["last_price"])


def weekly_closing(ticker_lst, start, end):
    full_data = yf.download(ticker_lst, start, end, progress=False, auto_adjust=True)["Close"]
    return full_data.resample("W").last()


def get_history(ticker_lst, start, end):
    """
    
    """
    data = yf.download(ticker_lst, start, end, interval='1d', auto_adjust=False, progress=False)
    prices = data['Adj Close'].copy()
    vols = data["Volume"].copy()
    return prices, vols

def monthly_filter_avg_volume(vol_series):
    return 

def get_risk_free_rate():
    """
    Gets Canadian risk free rate using Tbills 
    """
    #yfinance doesn't support the canadian tbills so this is commented out
    #ticker = yf.Ticker("IR3TCD01CAM156N") # canadian tbill
    #data = ticker.history(period='1d')["Close"].iloc[-1]
    #return (data / 100)
    return 0.0218 # taken from https://ycharts.com/ for Canada 3 Month Treasury Bill Yield

def get_sharpe_ratio(t, data):
    """
    Inputs: 
        - t (ticker)
        - data (closing data for ticker t)
    Outputs:
        - sharpe (sharpe ratio of data)
    
    Function pulls RF rate and calculates daily return percentage
    Calculates the Sharpe Ratio of data.
    """
    risk_free_rate = get_risk_free_rate() # closest to US treasury bills 2024-2025, might want to make a function to auto calc it
    
    daily_returns = data.pct_change().dropna()
    annual_return = (1 + daily_returns.mean())**252 - 1
    annual_volatility = daily_returns.std() * np.sqrt(252)
    sharpe = (annual_return - risk_free_rate) / annual_volatility
    return sharpe

def get_sortino_ratio(t, data):
    """
    Inputs: 
        - t (ticker)
        - data (closing data for ticker t)
    Outputs:
        - sortino (sortino ratio of data)

    Function pulls RF rate and calculates the daily return percentage.
    Removes the positive daily returns and calculates the downside deviation.
    Calculates sortino ratio.
    """
    risk_free_rate = get_risk_free_rate()
    daily_returns = data.pct_change().dropna()
    #filter to only downside returns
    downside = daily_returns[daily_returns < 0]
    
    if len(downside) < 10: #we can change this threshold later
        return -np.inf
    # annualized return
    annual_return = (1 + daily_returns.mean())**255 - 1
    # annualized downside deviation
    downside_deviation = downside.std() * np.sqrt(252)

    # calculate sortino ratio
    sortino = (annual_return - risk_free_rate) / downside_deviation

    return sortino

def get_MDD(t, data):
    """
    Inputs: 
        - t (ticker)
        - data (closing data for ticker t)
    Outputs:
        - MDD (Non absoluted value of max drawdown percentage of data)

    Function keeps track of the running maximum stock price and compares the current price to the max peak. 
    Returns the minimum (most negative) comparison
    """
    running_max = data.cummmax() #takes the up to date maxiumum (peak)

    drawdown = (data - running_max) / running_max #calc percentage drop from peak
    MDD = drawdown.min() # get the largest drop percentage
    return MDD

def get_momentum(t):
    """
    Inputs: 
        - t (ticker)
        - data (closing data for ticker t)
    Outputs:
        - Momentum (Returns the momentum of data compared to 6 months ago) 
    """
    closing_data = yf.download(t, start= "2025-03-01", end=LOOK_BACK_END, progress=False)["Close"]
    init_price = closing_data.iloc[0]
    final_price = closing_data.iloc[-1]
    momentum = final_price / init_price
    return momentum


In [ ]:
#PORTFOLIO CONSTRUCTION
def possible_companies(ticker_lst, FX):

    all_prices, Volume_all = get_history(ticker_lst, "2024-10-01", "2025-09-30")

    pos_companies = {}

    for t in ticker_lst:
        try: # filter out uneligible stocks
            # check for average daily volume to meet rule requirements
            vol_ser = Volume_all.get(t)
            avg_volume = monthly_filter_avg_volume(vol_ser)
            last_close = float()
            if avg_volume < MIN_avg_daily_volume:
                continue


            t_obj = yf.Ticker(t)   
            info = t_obj.get_info()
            sector = info.get("industry")
            mcap = info.get("marketCap")
            currency = info.get("currency")
            last_close = float(t_obj.fast_info["last_price"])

            closing_data = yf.download(t, start=LOOK_BACK_START, end=LOOK_BACK_END, progress=False)["Close"]
            # Add sortino here
            sortino = get_sortino_ratio(t, closing_data)
            # Add sharpe here
            sharpe = get_sharpe_ratio(t, closing_data)
            # Add Max Drawdown here
            MDD = get_MDD(t, closing_data)
            # Add momentum here
            momentum = get_momentum(t)


            pos_companies[t] = {
                'Ticker' : t,
                'Currency' : currency,
                'Sector' : sector,
                'Market Cap (cad)' : mcap,
                'Last Price' : last_close,
                # Add the data analysis here
                'Sortino' : sortino,
                'Sharpe' : sharpe,
                'Max DrawDown' : MDD,
                'Momentum' : momentum,
            }

        except Exception:
            continue # in case a step errors

    return pos_companies


In [ ]:
def build_port(ticker_lst):
    
    
    FX = get_exchange_rate()

    candidates = possible_companies(ticker_lst, FX)
    return


#tech_data =yf.download(ticker_list, start, end, progress=False, auto_adjust=True)["Close"]

#t_list = get_tickers("Tickers_file.csv")
#print(t_list)
#info_dict = get_info(t_list)
#df1 = pd.DataFrame(info_dict)
#weekly_df = weekly_closing(t_list, "2022-01-01", "2025-08-08")
#prices_all, vols_all = get_history(t_list, LOOK_BACK_START, LOOK_BACK_END)
#print(vols_all)
#print(prices_all)
#ex = get_exchange_rate()
#print(ex)


: 

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Nelson Kang, Mane Muradyan, Phelan Niu


In [ ]:

closing_data = yf.download("AMZN", start=LOOK_BACK_START, end=LOOK_BACK_END, progress=False)["Close"]
sharpe = get_sharpe_ratio("AMZN", closing_data)
sortino = get_sortino_ratio("AMZN", closing_data)
MDD = get_MDD('AMZN', closing_data)
momentum = get_momentum("AMZN", closing_data)
print(f"sharpe: {sharpe} sortino: {sortino} MDD: {MDD} Momentum: {momentum}")